# Scikit Learn Benzene Price Forecasting

## 0.0 Notes and Explainations

This notebook uses SQL to query::
* lyb-sql-devdacore-002.5bff9fcb8330.database.windows.net 
    * To extract the information contained in the t-code ZMRRELPO
* lyb-sql-prddacore-002.bed79ae4ef8b.database.windows.net
    * To extract ZEMA IHS data
    * To extract ZEMA ICIS data

### 0.1 Environment setup

This workbook utilizes the py37_benzene environment which can be installed via the Anaconda Prompt from your local repo sync by running:
> conda env create -f py37_benzene.yml

This .yml file is stored in the root scripts directory

## 1.0 Prepare Workspace

### 1.1 Import Standard Libraries and configure runtime parameters

In [1]:
##
# Import Basic Python DS Libraries 
import pandas as pd     # Standard data science package
import numpy as np      # Additional numerical functions
#import dtale
import warnings
warnings.filterwarnings('ignore')

##
# Import Advanced Python DS Libraries
import missingno as msno    # Missing value toolbox
#from pandas_profiling import ProfileReport     # Integrated/deep reporting, resource intensive
from statsmodels.tsa.stattools import adfuller  # Statistical test for stationary data

##
# Import Database Connection Libraries
#import pyodbc           # Database connector

##
# Import Plotting Libraries
import matplotlib.pyplot as plt                # Full featured plotting toolbox
plt.rcParams['figure.figsize'] = (25,8)
#import plotly.graph_objects as go               #Plotly GO toolbox
import plotly.express as px                    # Plotly Express plotting toolbox
import plotly.io as pio                        # Addtional Controls for plotly to allow visuals within VSCode Notebook
#import seaborn as sns       # Seaborn plotting toolbox

##
# Import Operating System Interface Libraries
import os       # operating system interface
import sys
#import calendar
#import datetime as dt
from datetime import datetime, timedelta, date

##
# Import File Format Libraries
#import pyarrow.feather as feather       # Advanced datasource import/export toolbox (Apache Parquet)
import shelve # Allows serialization (pickle-ing) of multiple opjects and saving them to dict objects for later use
import sqlite3
##
# Import ML and Forecasting Libraries
#from scipy import interpolate
#from sklearn import metrics     # Metrics for sklearn ML models
from sklearn.ensemble import RandomForestRegressor # ML Model package
#from sklearn.tree import DecisionTreeRegressor  # ML Model package

from sktime.forecasting.model_selection import ExpandingWindowSplitter, SlidingWindowSplitter
#from sklearn.model_selection import TimeSeriesSplit

#from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import RFECV, RFE
#from xgboost import XGBRegressor, XGBRFRegressor
#from prophet import Prophet

##
# Setup packages for sending messages
import requests
import json

##
# Import timing function(s)
from time import perf_counter
import datetime

In [2]:
# Configure Libraries
pio.renderers.default = "notebook_connected" # Configure plotly to print within VSCode environment
#pio.renderers.default = "vscode"


In [3]:
# Configure Run Environment
AllLPC = True # Used in the "Deal with non-Stationary data" step to ensure that e take the Percent Change for every column and not just those that are non-stationary

### 1.2 Import Custom Functions

In [4]:
# Define location for custom functions
module_path = os.path.abspath(os.path.join('../Functions'))

# Verify it's accessible for loading
if (module_path not in sys.path) & (os.path.isdir(module_path)):
    sys.path.append(module_path)
    print('Added', module_path, 'to system paths')

elif (module_path in sys.path) & (os.path.isdir(module_path)):
    print(module_path, 'ready to be used for import')

else:
    print(module_path, 'is not a valid path')

# Import Custom Functions
try: from multi_plot import *; print('Loaded multiplot')
except: print('failed to load multi_plot')

try: from StationaryTools import *; print('Loaded StationaryTools')
except: print('Failed to load StationaryTools')

try: from RegressionTools import *; print('Loaded RegressionTools')
except: print('Failed to load RegressionTools')

try: from NaiveForecasting import *; print('Loaded NaiveTools')
except: print('Failed to load NaiveTools')

try: from ModelingTools import *; print("Loaded ModelingTools")
except: print('ModelingTools failed to load')

try: from  sk_ts_modelfit import *; print('Loaded sk_ts_modelfit')
except: print('Failed to load sk_ts_modelfit')

Added c:\Users\baanders\Documents\Benzene Forecasting\Scripts\Functions to system paths
Loaded multiplot
Loaded multiplot
Loaded StationaryTools
Loaded StationaryTools
Loaded RegressionTools
Failed to load NaiveTools
Loaded ModelingTools
Loaded sk_ts_modelfit


### 1.3 Configure messaging

In [5]:
url = 'https://lyondellbasell.webhook.office.com/webhookb2/0b44f724-4c38-4322-af43-1cc8a79b2240@fbe62081-06d8-481d-baa0-34149cfefa5f/IncomingWebhook/7d6ffad65b414df9bd9bf5fa64da44b2/76b3b453-0a85-40ff-a5d7-15d042decdf6'

#payload = {
#    "text": "Sample alert text"
#}
#headers = {
#    'Content-Type': 'application/json'
#}
#response = requests.post(url, headers=headers, data=json.dumps(payload))
#print(response.text.encode('utf8'))

## 2.0 Import Data

In [6]:
# Define default storage location for files
dataroot = '../../Data/Parquet/SKLearn Data/'
ifilename = '20220426_weekly_for_modeling'

In [7]:
# Check if data location above exists. If it does import dataset.
# All datasets imported with name df so that we can generically 

if os.path.isdir(dataroot):
    df = pd.read_parquet(dataroot+ifilename+'.parquet')
    print(ifilename + ' dataset loaded with shape', df.shape, 'and', df.isna().sum().sum(), 'NaN values')
    
else:
    print('Storage location does not exist. Please update directory and try again.')

20220426_weekly_for_modeling dataset loaded with shape (369, 5306) and 0 NaN values


## 4.0 Modeling

### 4.1 Limit Columns to be used in fits

In [8]:
print('Shape before droping columns', df.shape)
df = df.loc[:,~df.columns.str.contains('^USD')]
df = df.loc[:,~df.columns.str.contains('^....USD')]
df = df.loc[:,~df.columns.str.contains('-CLOSE')]
df = df.loc[:,~df.columns.str.contains('-HIGH')]
df = df.loc[:,~df.columns.str.contains('-HIGHLOW2')]
df = df.loc[:,~df.columns.str.contains('-LOW')]

print('Shape after droping columns', df.shape)

Shape before droping columns (369, 5306)
Shape after droping columns (369, 1568)


#### 4.1.1 Scikit Learn Modeling Function

#### 4.1.2 Configure Global Model Inputs

In [9]:
# Simple utility to find a column name based on what it starts with
# May need it to find target column values

[col for col in df if col.startswith('Benzene-Spot, Current Month, High')]

['Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_1',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_2',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_3',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_4',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_5',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_6',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_7',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_8',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America_lag_9',
 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon

In [10]:
# Deal with inf and -inf values before train/test splitting
inf_vals = df.isin([np.inf, -np.inf]).sum().sum()
nan_vals = df.isna().sum().sum()
print('There are:',inf_vals,'inf or -inf values in df')
print('There are:',nan_vals,'NaN values in df')
if (inf_vals>0) or (nan_vals>0): 
    df = df.replace([np.inf, -np.inf], np.NaN)
    df = df.dropna(axis=0)
    inf_vals = df.isin([np.inf, -np.inf]).sum().sum()
    nan_vals = df.isna().sum().sum()
    print('After conversion there are:', inf_vals,'inf or -inf values in df')
    print('After conversion there are:',nan_vals,'NaN values in df')

There are: 0 inf or -inf values in df
There are: 0 NaN values in df


In [11]:
# Our target and Identifying columns won't change for any of the models so we'll define them once here
#target = 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America'
target = 'Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America'

# Our Identifying (IDcol) is 'date' for time series analyses
IDcol = ['date']

# All columns that are not Targets(predictee's) or ID's(dates) should be used to predict the Target
predictors = [x for x in df.columns if x not in [target]+[IDcol]]

print("Algorithm will attempt to predict:\n\t", target, "\nusing:\n\t", IDcol, "\nbased on:\n\t", len(predictors), "predictors")

Algorithm will attempt to predict:
	 Benzene-Spot, Current Month, High-N/A-Cents per Gallon-FOB Houston, TX-North America 
using:
	 ['date'] 
based on:
	 1567 predictors


#### 4.1.2 Test for correlations

In [ ]:
#df_corr = df.corr()
#keep = np.triu(np.ones(df_corr.shape)).astype('bool').reshape(df_corr.size)
#df_corr_long = df_corr.stack()[keep]
#df_corr_long[df_corr_long>0.8]

In [ ]:
#correlated_features = set()
#correlation_matrix = df.drop(target, axis=1).corr()

#for i in range(len(correlation_matrix.columns)):
#    for j in range(i):
#        if abs(correlation_matrix.iloc[i, j]) > 0.8:
#            colname = correlation_matrix.columns[i]
#            correlated_features.add(colname)

#print('# Features in df:',len(df.columns),'\n# correlated features:',len(correlated_features))

# Remove any columns that are correleated
#df.drop(columns=correlated_features, inplace=True)
#predictors = [x for x in df.columns if x not in [target]+[IDcol]]

#### 4.1.3 Extract prefered time frame and lag for prediction

In [ ]:
# Roll back if the below has been run and needs to be changed
#df = df_full.copy(deep=True)


In [ ]:
# Set start_date for slicing df
start_date = '2015-01-01'
# Set end_date for slicing df
end_date = '2018-12-31'#'2019-12-31'

#df_full = df.copy(deep=True)
df = df[start_date : end_date]
print('Before processing:\ndf starts on:', df.index[0],'\n\tends on:', df.index[-1])

# Loop counter used for iterative lagging
lcnt = 1 # 4, 8, 12

# Add periods to end of dataframe equal to amount of fshift
#future = pd.DataFrame(pd.date_range(df.index[-1], periods = lcnt, freq='w')).set_index(0)
#future = future.iloc[1:,:]
#df = pd.concat([df, future], axis=0)

# Shift predictor values forward
df[predictors] = df[predictors].shift(lcnt)
df = df.dropna(axis=0)

# Remove top _fshift_ rows that are now NaN
#df = df.iloc[fshift:,:]

# Create dataframe for predictions:
#df_pred = df[predictors].iloc[-fshift:,:]

# Remove prediction from df
#df = df.iloc[:-fshift,:]

print('After processing:\ndf starts on:', df.index[0],'\n\tends on:', df.index[-1])
#print('\nfuture starts on:', future.index[0],'\n\tends on:', future.index[-1])

### 4.2 Scikit Learn Modeling

In [ ]:
# Define alg search params
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 50)] # [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt'] # ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 20)] # [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 11)] #[2, 5, 10] # [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4, 5, 10, 20] # [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False] # [True, False]
# Define Random Forest Regressor HP grid for DV
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
               }

In [ ]:
# RFE returned 1398 columns using thise
init_params = { 'n_estimators': 787,
                'min_samples_split': 6,
                'min_samples_leaf': 10,
                'max_features': 'sqrt',
                'max_depth': 41,
                'bootstrap': True}

# Final pass of search algorithm found this with only 60 RFE columns
init_params = {'n_estimators': 714,
 'min_samples_split': 11,
 'min_samples_leaf': 5,
 'max_features': 'sqrt',
 'max_depth': 62,
 'bootstrap': False}

In [ ]:
# This function works as expected, but the RFE rarely limits the number of columns very agressively.
#df_rfe, search_alg, rfe_alg, rfe_col_log, rf_params_log = RF_RFE_RandomSearchCV(df, 13*3, 13, 13, 60, random_grid, predictors, target, max_iter=10, init_params=None, verbose=True, debugging=False)

In [ ]:
#rf_params_log#[9].to_dict()

#### 4.2.0 Recursive Feature Elimination

#### 4.2.0 RFE CV

In [ ]:
# Define algorithm for modeling
rf_params = {'n_estimators': 714,
 'min_samples_split': 11,
 'min_samples_leaf': 5,
 'max_features': 'sqrt',
 'max_depth': 62,
 'bootstrap': False,
    'verbose':0,
    'n_jobs':-1
    }

# Define initial lcnt for multi-index storing of parameters during runs
lcnt = 0

# Define loggind data frames
rf_params_log = pd.DataFrame()
rfe_col_log = pd.DataFrame()
rfe_cv_log = pd.DataFrame()
rfe_fi_log = pd.DataFrame()


Start Looping iterations below here, don't re-run the above or you'll be starting all over!

In [ ]:
df.shape

In [ ]:
# To use when not letting RFE run but not wanting to comment and affect code
feat_count = df.shape[1]
#feat_count = 60

# create cv_iterable for this run
cv_iterables = ts_cv_exwindow(df, 13*3, 13, 13, verbose=True)

# Using generated rf_params (basic to start and then updated after each SearchCV run)
alg = RandomForestRegressor(**rf_params)
#alg = XGBRegressor()
#alg = XGBRFRegressor()

rfe_alg = RFECV(
            alg, 
            step=0.1, 
            min_features_to_select=feat_count, 
            cv=cv_iterables, 
            scoring='neg_root_mean_squared_error',
            verbose=0, 
            n_jobs=-1, 
            importance_getter='auto'
            )
rfe_alg
# None = returns 60, no cv_result
# r2 = returns 60, no cv_result
# neg_root_mean_squared_error = returns 1000+ with cv_result
# neg_mean_absolute_error = returns 1000+ with cv_result
# neg_mean_squared_error = returns 1000+ with cv_result
# max_error = returns 1000+ with cv_result
# neg_mean_absolute_percentage_error = returns 1000+ with cv_result


In [ ]:
# Get time, in seconds, before optimization starts (tic)
tic = perf_counter()

# Run CV optimization
rfe_out = rfe_alg.fit(df[predictors], df[target])

# Extract Selected features and information about them (rank and if included in output)
#df_features = pd.DataFrame(columns = ['feature', 'selected', 'ranking'])
print(df.shape[1])
#for i in range(df[predictors].shape[1]):
#    print(i, end='\r')
#    row = {'feature': df.columns[i], 'selected': rfe_out.support_[i], 'ranking': rfe_out.ranking_[i]}
#    df_features = df_features.append(row, ignore_index=True)
df_features = pd.DataFrame({'feature':df[predictors].columns,'selected':rfe_out.support_, 'rank':rfe_out.ranking_})

df_rfe = df[df.columns[rfe_out.get_support(1)]]
print('df_rfe shape: ', df_rfe.shape)

# Get time, in seconds, after optimization finishes (toc)
toc= perf_counter()

# Calculate run_time in seconds
run_time = toc-tic
print(str(timedelta(seconds=run_time)))

# Send a Teams message using the Benzene Forecasting Team Incoming Webhook with output info from the model
payload = {
    "text": "RFE CV search complete. <br><br> \
        Run time:<br>&nbsp;&nbsp;" + str(timedelta(seconds=run_time)) + "<br><br> \
        Selected Columns:<br>" + pd.DataFrame(df_rfe.columns)[0:59].to_html() # +  "<br><br> \
}
headers = {
    'Content-Type': 'application/json'
}
#response = requests.post(url, headers=headers, data=json.dumps(payload))
#print(response.text.encode('utf8'))

In [ ]:
new_rfe_fi = pd.DataFrame({'feature':df_rfe.columns, lcnt:rfe_out.estimator_.feature_importances_}).set_index('feature')
#rfe_fi_log = pd.concat([rfe_fi_log, new_rfe_fi], axis=1)
new_rfe_fi[0].sort_values(ascending=False)
#rfe_fi_log

In [ ]:
pd.DataFrame.from_dict(rfe_out.cv_results_)

In [ ]:
pd.DataFrame.from_dict(rfe_out.cv_results_)

In [ ]:
# Add current run to rfe_cv_log
new_cv = pd.concat([pd.DataFrame.from_dict(rfe_out.cv_results_)], keys=[lcnt+1], names=['iteration', 'data'], axis=1)
rfe_cv_log = pd.concat([rfe_cv_log, new_cv], axis=1)

# Add current run to rfe_col_log 
new_rfe = pd.concat([df_features.set_index('feature')], keys=[lcnt], names=['Iteration', 'Data'], axis=1)
rfe_col_log = pd.concat([rfe_col_log, new_rfe], axis =1)
rfe_col_log
#rfe_cv_log

In [ ]:
rfe_out.get_support(0).sum()

#### 4.2.1 Hyper Parameter Optimization Cross Validataion

#### Looped lagging Hyperparameter Optimization with storage

In [12]:
# Functionalized RFE + HP Tuning

def Lag_RF_RandomSearchCV(df, iw, sl, fh, random_grid, pred, targ, fit_shelf, lags=13, search_iters=10, init_params=None, verbose=True, debugging=False):
    """
    Iteratively lag df from 1 to 'lags' and perform RandomSearchCV to optimize each over the given interval
    Does NOT perform any prediction work, that will be done seperately this only optimizes the hyperparameters for the given lag with CV

    INPUT:
    RF_RFE_RandomSearchCV performs RFE and RandomSearch with CV to optimize RandomForestRegression()
    df = dataframe to be processed
    iw = initial window for CV
    sl = step length for CV
    fh = forecast horizion for CV
    random_grid = dictionary of grid options to search during hyperparameter optimizataion
    pred = list of predictors to use in first loop
    targ = target column to be predicted
    search_iters = number of iterations to search within random_grid (default=10)
    init_params = initial RandomForest parameters used when peforming run 0 RFE (default=None)
    verbose = controls standard print during run (default=True)
    debuggin = controls debugging print during run (default=False)
    
    OUTPUT:
    df_rfe = Last RFE reduced dataframe
    search_alg = last output of RandomSearchCV optimization Model
    rfe_alg = last output of RFECV model
    rfe_col_log = Log of all RFE removed columns in each iteration
    rf_params_log = log of all selected hyperparameter sets in each iteration
    """
    # Configure initial RandomForestRegression Parameters
    if init_params:
        rf_params = init_params
    else:
        rf_params = {
            'verbose':0,
            'n_jobs':-1
        }
    # Open shelf to store fits
    s = shelve.open(fit_shelf, flag='c', writeback=True)
    if verbose: print(model_dataroot + model_shelf+' opened ')
    # Pre-Define data frames
    rf_params_log = pd.DataFrame()                    
    # Create and print cv data once
    cv_iterables = ts_cv_exwindow(df, iw, sl, fh, verbose=True)
    # Initialize fit parameter log
    df_fit_params = pd.DataFrame()
    # Initizlize cv_result log
    df_cv_results = pd.DataFrame()
    # Initialize run_time_log
    run_time_log = pd.DataFrame()
    # Store original df for recall later
    df_orig = df.copy(deep=True)
    # Set run_date in case optimizations spans multiple days
    run_date = date.today().strftime('%Y%m%d')
    
    #############################################
    # Loop through HyperParameter optimizations #
    #############################################
    tic_overall = perf_counter()
    for lcnt in range(1, lags+1, 1):
        print('\nLoop', str(lcnt))
        # Get time, in seconds, before iteration starts (tic)
        tic = perf_counter()
        ##############################
        # Perform Lagging Operations #
        ##############################
        # Revert df to original
        df = df_orig.copy(deep=True)
        # Shift prediction values forward lcnt steps
        df[pred] = df[pred].shift(lcnt)
        # Drop NaN values due to shift
        df = df.dropna(axis=0)
        if verbose: print('Shape of lag', lcnt, 'iteration df is', df.shape)
        ################################
        # Hyperparameter Optimizataion #
        ################################
        # create cv_iterable for this run
        cv_iterables = ts_cv_exwindow(df, iw, sl, fh, verbose=False)
        # Define algorithm for modeling
        alg = RandomForestRegressor(**rf_params)
        # Define a Random Search of fitting function that will sample the hyperparameter space and perform CV
        search_alg = RandomizedSearchCV(  # Original values were: cv=3 , n_iter=100
            estimator = alg, # Defined above, RandomForestRegression or XGBoost
            param_distributions = random_grid, # Grid or distribution of hyperparameters to sample
            n_iter = search_iters, # How many iterations of param_distribution will be sampled
            cv = cv_iterables, # Iterable ist of CV space created by ExpandingWindowSplitter for TS
            verbose=1, # High = more printing, 0=No Print, 10=all/max?
            #random_state=42, # Forced random number state to allow comparison between runs
            n_jobs = -1, # How many cores, -1 = All available
            return_train_score = True, # Computationally expensive, but returns training scores to over/under fit comparison of CV space
            refit = True, # will allow the use of the best estimator with .predict() after run
            scoring='neg_mean_absolute_error' # closest to 0 is best fit
            )
        # Run CV optimization
        fit_out = search_alg.fit(df[pred], df[targ])
        # Pickle and store current model on shelf
        s[run_date+"_lag_"+str(lcnt)] = fit_out
        # Log current iteration best_params
        new_fit_param = pd.DataFrame.from_dict(fit_out.best_params_, orient='index').T
        new_fit_param['run_date'] = run_date
        new_fit_param['lag'] = lcnt
        df_fit_params = pd.concat([df_fit_params, new_fit_param], axis=0)
        # Log current iteration cv_results_
        new_cv_result = pd.DataFrame.from_dict(fit_out.cv_results_)
        new_cv_result['run_date'] = run_date
        new_cv_result['lag'] = lcnt
        new_cv_result = new_cv_result.reset_index().rename(columns={'index':'CV_Fold'})
        df_cv_results = pd.concat([df_cv_results, new_cv_result], axis=0, ignore_index=False)
        # Log current fit runtime from tic-toc
        if verbose: print('Optimization runtime was',str(timedelta(seconds=perf_counter()-tic)))
        new_run_time = pd.DataFrame([perf_counter()-tic]).rename(columns={0:'run_time'})
        new_run_time['run_date'] = run_date
        new_run_time['lag'] = lcnt
        run_time_log = pd.concat([run_time_log, new_run_time], axis=0, ignore_index=False)
    # Log total function run_time as lag=0 value
    total_run_time = perf_counter() - tic_overall
    new_run_time['run_time'] = total_run_time
    new_run_time['run_date'] = run_date
    new_run_time['lag'] = 0
    run_time_log = pd.concat([run_time_log, new_run_time], axis=0, ignore_index=True)
    # Close (and sync) the shelf
    s.close()
    # Send a Teams message using the Benzene Forecasting Team Incoming Webhook with output info from the model
    payload = {
        "text": "Hyperparameter search with lagging complete. <br><br> \
            Run time:<br>&nbsp;&nbsp;" + str(timedelta(seconds=total_run_time)) + "<br><br> \
            Total Lags:<br>" + str(lcnt) # +  "<br><br> \
    }
    headers = {
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    #print(response.text.encode('utf8'))
    print('Total Runtime:', str(timedelta(seconds=total_run_time)),' H:M:S.sss')
    return df_fit_params, df_cv_results, run_time_log

In [13]:
# Run parameters for function
model_dataroot = '../../Data/Models/'
model_shelf = 'Random_Forest_Models'
fit_shelf = model_dataroot+model_shelf

# Roll back if the below has been run and needs to be changed
#df = df_full.copy(deep=True)

# Set start_date for slicing df
start_date = '2015-01-01'
# Set end_date for slicing df
end_date = '2018-12-31'#'2019-12-31'

#df_full = df.copy(deep=True)
df = df[start_date : end_date]
df.shape

(195, 1568)

In [15]:
# Define alg search params
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 50)] # [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt'] # ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 20)] # [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 11)] #[2, 5, 10] # [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4, 5, 10, 20] # [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False] # [True, False]
# Define Random Forest Regressor HP grid for DV
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
               }

In [ ]:
# Determine where to start training for cv interval
init_ind = df.index.get_loc('2017-12-31')
cv_iterables = ts_cv_exwindow(df, init_ind, 1, 13, verbose=True)

In [16]:
fit_params, cv_res_log, runtimelog = Lag_RF_RandomSearchCV(df, 142, 1, 13, random_grid, predictors, target, fit_shelf, lags=13, search_iters=60, init_params=None, verbose=True, debugging=False)

../../Data/Models/Random_Forest_Models opened 
Initial training window ends on:  2017-12-31 00:00:00+00:00
Number of Folds = 41

Loop 1
Shape of lag 1 iteration df is (194, 1568)
Fitting 40 folds for each of 60 candidates, totalling 2400 fits
Optimization runtime was 0:44:09.831187

Loop 2
Shape of lag 2 iteration df is (193, 1568)
Fitting 39 folds for each of 60 candidates, totalling 2340 fits
Optimization runtime was 0:59:10.680695

Loop 3
Shape of lag 3 iteration df is (192, 1568)
Fitting 38 folds for each of 60 candidates, totalling 2280 fits
Optimization runtime was 1:08:13.427673

Loop 4
Shape of lag 4 iteration df is (191, 1568)
Fitting 37 folds for each of 60 candidates, totalling 2220 fits
Optimization runtime was 1:05:18.047236

Loop 5
Shape of lag 5 iteration df is (190, 1568)
Fitting 36 folds for each of 60 candidates, totalling 2160 fits
Optimization runtime was 0:45:42.653456

Loop 6
Shape of lag 6 iteration df is (189, 1568)
Fitting 35 folds for each of 60 candidates, to

In [20]:
#fit_params.loc[(fit_params['run_date'] == '20220427') & (fit_params['lag']==2)]
fit_params

,n_estimators,min_samples_split,min_samples_leaf,max_features,max_depth,bootstrap,run_date,lag
0,1228,50,3,auto,104,True,20220427,1
0,604,45,5,auto,10,True,20220427,2
0,1044,6,4,sqrt,73,False,20220427,3
0,971,6,1,sqrt,94,False,20220427,4
0,236,6,3,auto,73,False,20220427,5
0,1522,16,10,auto,31,False,20220427,6
0,1191,2,2,sqrt,57,True,20220427,7
0,346,26,20,sqrt,67,True,20220427,8
0,1889,50,1,sqrt,88,True,20220427,9
0,1302,2,20,sqrt,62,True,20220427,10


In [19]:
cv_res_log.tail(30)

,CV_Fold,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,...,split34_train_score,split35_train_score,split36_train_score,split37_train_score,split38_train_score,split39_train_score,mean_train_score,std_train_score,run_date,lag
30,30,5.183449,1.165719,0.524160,0.127320,1155,11,2,sqrt,94,...,NaN,NaN,NaN,NaN,NaN,NaN,-13.578478,0.483357,20220427,13
31,31,3.289010,0.608161,0.357179,0.141608,567,2,10,sqrt,None,...,NaN,NaN,NaN,NaN,NaN,NaN,-69.169215,3.322206,20220427,13
32,32,4.285176,0.687858,2.081352,4.187246,1375,50,3,sqrt,62,...,NaN,NaN,NaN,NaN,NaN,NaN,-70.275018,7.860413,20220427,13
33,33,102.726422,31.000900,49.501191,28.094640,2000,40,10,auto,41,...,NaN,NaN,NaN,NaN,NaN,NaN,-71.953680,6.017911,20220427,13
34,34,9.111480,13.479852,2.472898,1.735500,751,45,10,sqrt,62,...,NaN,NaN,NaN,NaN,NaN,NaN,-108.388009,6.388787,20220427,13
35,35,17.952647,7.512542,13.412667,6.249237,567,26,20,auto,78,...,NaN,NaN,NaN,NaN,NaN,NaN,-101.687606,4.059966,20220427,13
36,36,43.787848,9.336421,23.055841,13.155508,1375,16,20,auto,78,...,NaN,NaN,NaN,NaN,NaN,NaN,-101.687606,4.059966,20220427,13
37,37,10.303610,13.389004,3.739020,7.606681,604,16,4,sqrt,41,...,NaN,NaN,NaN,NaN,NaN,NaN,-21.813726,0.657206,20220427,13
38,38,5.553453,0.750828,0.550164,0.039428,1742,2,20,sqrt,78,...,NaN,NaN,NaN,NaN,NaN,NaN,-79.041645,3.858449,20220427,13
39,39,6.308904,0.601813,0.560379,0.109642,1669,50,2,sqrt,73,...,NaN,NaN,NaN,NaN,NaN,NaN,-70.220093,8.032447,20220427,13


In [17]:
runtimelog

,run_time,run_date,lag
0,2649.831570,20220427,1
1,3550.681066,20220427,2
2,4093.428042,20220427,3
3,3918.047604,20220427,4
4,2742.653823,20220427,5
5,3185.202061,20220427,6
6,3395.335193,20220427,7
7,2989.001670,20220427,8
8,2336.086703,20220427,9
9,2766.902851,20220427,10


In [ ]:
# Define default storage location for shelve of models
model_dataroot = '../../Data/Models/'
model_shelf = 'Random_Forest_Models.db'
#model_shelf = date.today().strftime('%Y%m%d')+'_Shelved_Models.db'

# Open shelve object on first run
if os.path.isdir(model_dataroot):
    s = shelve.open(model_dataroot+model_shelf, flag='c', writeback=True)
    print(model_dataroot + model_shelf+' opened ')
else:
    os.mkdir(model_dataroot)
    print('Shelve storage location did not exist. It has been created\n')
    s = shelve.open(model_dataroot+model_shelf, flag='c', writeback=True)
    print(model_dataroot + model_shelf+' opened ')

In [ ]:
df_rfe = df.copy(deep=True)

# create cv_iterable for this run
cv_iterables = ts_cv_exwindow(df_rfe, 13*3, 13, 13, verbose=True)

# Define algorithm for modeling
# Use no bias when beginnng the search so do not include rf_params
alg = RandomForestRegressor(verbose=0, n_jobs=-1) #**rf_params)

# Define alg search params

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 50)] # [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt'] # ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 20)] # [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 11)] #[2, 5, 10] # [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4, 5, 10, 20] # [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False] # [True, False]

# Define Random Forest Regressor HP grid for DV
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
               }

# Define rg_str which is a readable version of random_grid used in json payload for Teams notification
rg_str = '<br>&nbsp; n_estimators:'+ str(n_estimators) + \
               '<br>&nbsp; max_features: '+ str(max_features) +\
               '<br>&nbsp; max_depth: ;'+ str(max_depth) + \
               '<br>&nbsp; min_samples_split: '+ str(min_samples_split) + \
               '<br>&nbsp; min_samples_leaf: '+ str(min_samples_leaf) + \
               '<br>&nbsp; bootstrap: '+ str(bootstrap)

print(random_grid)
#print(rg_str)

In [ ]:
# Define a Random Search of fitting function that will sample the hyperparameter space and perform CV
# """
search_alg = RandomizedSearchCV(  # Original values were: cv=3 , n_iter=100
    estimator = alg, # Defined above, RandomForestRegression or XGBoost
    param_distributions = random_grid, # Grid or distribution of hyperparameters to sample
    n_iter = 2, # How many iterations of param_distribution will be sampled
    cv = cv_iterables, # Iterable ist of CV space created by ExpandingWindowSplitter for TS
    verbose=1, # High = more printing, 0=No Print, 10=all/max?
    #random_state=42, # Forced random number state to allow comparison between runs
    n_jobs = -1, # How many cores, -1 = All available
    return_train_score = True, # Computationally expensive, but returns training scores to over/under fit comparison of CV space
    refit = True, # will allow the use of the best estimator with .predict() after run
    scoring='neg_mean_absolute_error' # closest to 0 is best fit
    ) 
"""
# Define a Grid Search of fitting function that will sample the hyperparameter space and perform CV
search_alg = GridSearchCV(  # Original values were: cv=3 , n_iter=100
    estimator = alg, # Defined above, RandomForestRegression or XGBoost
    param_grid = random_grid, # Grid or distribution of hyperparameters to sample
    cv = cv_iterables, # Iterable ist of CV space created by ExpandingWindowSplitter for TS
    verbose=1, # High = more printing, 0=No Print, 10=all/max?
    n_jobs = -1, # How many cores, -1 = All available
    return_train_score = True, # Computationally expensive, but returns training scores to over/under fit comparison of CV space
    refit = True, # will allow the use of the best estimator with .predict() after run
    scoring='neg_mean_absolute_error', # closest to 0 is best fit  
    ) 
"""
print(search_alg)

In [ ]:

# Get time, in seconds, before optimization starts (tic)
tic = perf_counter()

# Run CV optimization
fit_out = search_alg.fit(df_rfe[predictors], df_rfe[target])

# Get time, in seconds, after optimization finishes (toc)
toc= perf_counter()

# Calculate run_time in seconds
run_time = toc-tic
print(str(timedelta(seconds=run_time)))

# Send a Teams message using the Benzene Forecasting Team Incoming Webhook with output info from the model
payload = {
    "text": "Grid search complete. <br><br> \
        Run time:<br>&nbsp;&nbsp;" + str(timedelta(seconds=run_time)) + "<br><br> \
        Best Score:<br>&nbsp;&nbsp;" + str(abs(fit_out.best_score_)) + "<br><br> \
        Best Params:<br>" + str(fit_out.best_params_).replace(',',',<br>&nbsp;&nbsp;') + '<br><br> \
        Grid Search Params:' + rg_str
}
headers = {
    'Content-Type': 'application/json'
}
#response = requests.post(url, headers=headers, data=json.dumps(payload))
#print(response.text.encode('utf8'))

In [ ]:
fit_out.best_params_

In [ ]:

df_fit_params = pd.DataFrame.from_dict(fit_out.best_params_, orient='index').T
df_fit_params['run_date'] = date.today().strftime('%Y%m%d')
df_fit_params['lag'] = 1
#df_fit_params = df_fit_params.set_index(['run_date'])

new_fit_param = pd.DataFrame.from_dict(fit_out.best_params_, orient='index').T
new_fit_param['run_date'] = date.today().strftime('%Y%m%d')
new_fit_param['lag'] = 2
#new_fit_param = new_fit_param.set_index(['run_date'])
df_fit_params = pd.concat([df_fit_params, new_fit_param], axis=0)

new_fit_param = pd.DataFrame.from_dict(fit_out.best_params_, orient='index').T
new_fit_param['run_date'] = '20220428'
new_fit_param['lag'] = 1
#new_fit_param = new_fit_param.set_index(['date'])
df_fit_params = pd.concat([df_fit_params, new_fit_param], axis=0)

new_fit_param = pd.DataFrame.from_dict(fit_out.best_params_, orient='index').T
new_fit_param['run_date'] = '20220428'
new_fit_param['lag'] = 2
#new_fit_param = new_fit_param.set_index(['date'])
df_fit_params = pd.concat([df_fit_params, new_fit_param], axis=0)

#df_fit_params = df_fit_params.set_index(['run_date', 'lag'])
#df_fit_params[slice('20220427')][slice(1,2)]

df_fit_params[(df_fit_params['run_date']==max(df_fit_params['run_date']))]# & (df_fit_params['lag']==2)]

In [ ]:
# Set rf_params to output for next loop
rf_params = fit_out.best_params_

# Log current iteration rf_params
best_param_df = pd.DataFrame.from_dict(fit_out.best_params_, orient='index')#.rename(columns={0:lcnt})

rf_params_log = pd.concat([rf_params_log, best_param_df], axis =1)
rf_params_log



In [ ]:
lcnt

In [ ]:
# Increment lcnt for next iteration
lcnt = lcnt + 1

In [ ]:
print("Best Estimator:\n\t", fit_out.best_estimator_)
print("\nBest Params:\n\t", fit_out.best_params_)
print("\nBest Index Model:\n\t", fit_out.best_index_)
#print("CV Results:", fit_out.cv_results_)

In [ ]:
# Predict the future values and compare them to the knowns
#df_predict = pd.DataFrame(df_predict).reindex_like(df_forecast)
df_predict = pd.DataFrame(fit_out.predict(df_forecast[predictors])).set_index(df_forecast.index).rename(columns={0:'prediction'})
df_predict = df_predict.shift(-fshift).dropna(axis=0)
df_predict = pd.concat([df_full[ df_predict.index[0] : df_predict.index[-1] ][target], df_predict], axis=1)
df_predict['pct_error'] = abs( df_predict[target] - df_predict['prediction']) / df_predict[target] * 100

# Send Teams Message of findings
payload = {
    "text": "Prediction complete. <br>" \
         + df_predict.to_html()
}
headers = {
    'Content-Type': 'application/json'
}
response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.text.encode('utf8'))


df_predict

### Dataset Organization and Testing

Shelve (Pickle)

Multi-index DataFrame --> Parquet

In [ ]:
run_time_log = pd.DataFrame()
#new_run_time=pd.DataFrame()
new_run_time = pd.DataFrame([22.45855]).rename(columns={0:'run_time'})
new_run_time['run_date'] = '20220427'
new_run_time['lag'] = lcnt
run_time_log = pd.concat([run_time_log, new_run_time], axis=0)

new_run_time = pd.DataFrame([24.5855]).rename(columns={0:'run_time'})
new_run_time['run_date'] = '20220427'
new_run_time['lag'] = lcnt+1
run_time_log = pd.concat([run_time_log, new_run_time], axis=0)

run_time_log

cv results

In [ ]:
cv_result_log = pd.DataFrame.from_dict(fit_out.cv_results_)
cv_result_log['lag'] = lcnt
cv_result_log

Store dictionary inside of DF

In [ ]:
entry1 = pd.DataFrame([['20220427',{'Key1':'Value1', 'key2':'value2'}]], columns=['date','params']).set_index('date')
entry2 = pd.DataFrame([['20220428',{'Key1':'Value1', 'key2':'value2'}]], columns=['date','params']).set_index('date')

param_log = pd.concat([entry1, entry2], axis=0)
param_log

In [ ]:
param_log[param_log.index == '20220428'].values

In [ ]:
dict1 = {'Key1':'Value1', 'key2':'value2'}
dict2 = {'Key3':'Value3', 'key4':'value4'}

test_df = pd.DataFrame()
test_df['First'] = dict1.items()
test_df['Second'] = dict2.items()
test_df

sqlite3 - Requires specific local driver in Power BI that isn't installable. Use Parquet instead for data

Shelve - will work great for storing models over time

In [ ]:
# Define default storage location for files
model_dataroot = '../../Data/Models/'
model_shelf = date.today().strftime('%Y%m%d')+'_Shelved_Models.db'

# Export Parquet file
if os.path.isdir(model_dataroot):
    # Export Data
    s = shelve.open(model_dataroot+model_shelf, flag='c', writeback=True)
    print(model_dataroot + model_shelf+' opened ')
else:
    os.mkdir(model_dataroot)
    print('Shelve storage location did not exist. It has been created\n')
    s = shelve.open(model_dataroot+model_shelf, flag='c', writeback=True)
    print(model_dataroot + model_shelf+' opened ')




In [ ]:
s['Fast Optimization3'] = fit_out
s.close()

In [ ]:
list(s)

In [ ]:
my_model = s['Fast Optimization']

In [ ]:
my_model.predict(df_pred)

# Documentation and Links

#### Time Series WFV and Optimization
* https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=24f6be86f95bfc1ec246dee7dcdd455e0a84a872&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c616e2d747572696e672d696e737469747574652f736b74696d652f323466366265383666393562666331656332343664656537646364643435356530613834613837322f6578616d706c65732f77696e646f775f73706c6974746572732e6970796e62&logged_in=false&nwo=alan-turing-institute%2Fsktime&path=examples%2Fwindow_splitters.ipynb&platform=android&repository_id=156401841&repository_type=Repository&version=98
* https://quantile.app/blog/cross_validation
* https://towardsdatascience.com/dont-use-k-fold-validation-for-time-series-forecasting-30b724aaea64
* https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
* https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/
* https://machinelearningmastery.com/what-is-bayesian-optimization/
* https://www.analyticsvidhya.com/blog/2021/05/bayesian-optimization-bayes_opt-or-hyperopt/
* https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html

#### Error
* https://towardsdatascience.com/forecast-kpi-rmse-mae-mape-bias-cdc5703d242d#:~:text=The%20Mean%20Absolute%20Percentage%20Error,average%20of%20the%20percentage%20errors.

#### SKTime Documentation:
* https://www.sktime.org/en/latest/examples/01_forecasting.html

#### SKforcast (PIP, not installed in environment):
* https://www.cienciadedatos.net/documentos/py27-time-series-forecasting-python-scikitlearn.html

#### Incremental Forecast loop using standard sklearn algorithms:
* https://www.analyticsvidhya.com/blog/2021/06/random-forest-for-time-series-forecasting/
* https://towardsdatascience.com/time-series-modeling-using-scikit-pandas-and-numpy-682e3b8db8d1
* https://www.ethanrosenthal.com/2019/02/18/time-series-for-scikit-learn-people-part3/

#### TS with Random Forest
* https://towardsdatascience.com/multivariate-time-series-forecasting-using-random-forest-2372f3ecbad1
* https://www.analyticsvidhya.com/blog/2020/03/beginners-guide-random-forest-hyperparameter-tuning/

#### ARIMA
* https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

#### Prophet 
* https://towardsdatascience.com/implementing-facebook-prophet-efficiently-c241305405a3